In [ ]:
import io
import numpy as np
import tensorflow_datasets as tfds
from google.colab import files
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)
train_data, test_data = imdb["train"], imdb["test"]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteBQPAY8/imdb_reviews-train.tfrecord…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteBQPAY8/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteBQPAY8/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
x_train = []
y_train = []
x_test = []
y_test = []

for sentence, label in train_data:
  x_train.append(sentence.numpy().decode("utf-8"))
  y_train.append(label.numpy())

for sentence, label in test_data:
  x_test.append(sentence.numpy().decode("utf-8"))
  y_test.append(label.numpy())

y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
VOCAB_SIZE = 5000
EMBEDDING_DIM = 16
MAX_LENGTH = 120
TRUNCATING = "post"


tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(x_train)

train_sequences = tokenizer.texts_to_sequences(x_train)
test_sequences = tokenizer.texts_to_sequences(x_test)

train_padded = pad_sequences(train_sequences, maxlen=MAX_LENGTH, truncating=TRUNCATING)
test_padded = pad_sequences(test_sequences, maxlen=MAX_LENGTH, truncating=TRUNCATING)

In [ ]:
model = Sequential()

model.add(layers.Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_length=MAX_LENGTH))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
model.fit(train_padded, y_train, epochs=10, validation_data=(test_padded, y_test))

Epoch 1/10
782/782 [==============================] - 6s 6ms/step - loss: 0.4945 - accuracy: 0.7413 - val_loss: 0.3920 - val_accuracy: 0.8196
Epoch 2/10
782/782 [==============================] - 5s 7ms/step - loss: 0.2740 - accuracy: 0.8888 - val_loss: 0.4065 - val_accuracy: 0.8214
Epoch 3/10
782/782 [==============================] - 4s 6ms/step - loss: 0.1387 - accuracy: 0.9570 - val_loss: 0.5182 - val_accuracy: 0.8017
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0439 - accuracy: 0.9922 - val_loss: 0.6319 - val_accuracy: 0.8000
Epoch 5/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0126 - accuracy: 0.9988 - val_loss: 0.7351 - val_accuracy: 0.7983
Epoch 6/10
782/782 [==============================] - 4s 6ms/step - loss: 0.0042 - accuracy: 0.9995 - val_loss: 0.8211 - val_accuracy: 0.8012
Epoch 7/10
782/782 [==============================] - 4s 6ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.8896 - val_accuracy: 0.8034
Epoch 

In [ ]:
word_index = tokenizer.word_index
index_word = dict([(value, key) for key, value in word_index.items()])

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]

out_m = io.open("meta.tsv", "w", encoding="utf-8")
out_v = io.open("vect.tsv", "w", encoding="utf-8")

for index in range(1, VOCAB_SIZE):
  word = index_word[index]
  out_m.write(word + "\n")

  embeddings = weights[index]
  out_v.write("\t".join([str(x) for x in embeddings]) + "\n")

out_m.close()
out_v.close()

In [ ]:
files.download("meta.tsv")
files.download("vect.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
sentence = "this movie is boring"

sequence = tokenizer.texts_to_sequences([sentence])
padded = pad_sequences(sequence, maxlen=MAX_LENGTH, truncating=TRUNCATING)

model.predict(padded, verbose=0)[0, 0]

0.005822703